# TP 2 : job d'entraînement, déploiement de modèle, prédiction par batch et en ligne, A/B Testing

<img src=https://www.headmind.com/wp-content/uploads/2024/01/logo_dark.png width="200">

<img src=https://www.ensta-paris.fr/profiles/createur_profil/themes/createur/dist/images/logo_ensta_new.jpg.pagespeed.ce.ERsGv8BS3M.jpg width="200">

### Objectifs

Au cours de ce TP, vous apprendrez à créer un modèle de Vision personnalisé, vous configurerez et lancerez un job d'entraînement, enfin, vous déploierez le modèle pour de la prédiction par lot (batch) puis pour de la prédiction en temps réel (streaming). 

On utilisera les services d'Azure Machine Leaning suivant : 
- Azure ML Données
- Azure ML Training
- Azure ML Models
- Azure ML Endpoints

## Liens utiles

- [Custom training](https://learn.microsoft.com/fr-fr/azure/machine-learning/how-to-train-model?view=azureml-api-2&source=recommendations&tabs=python) 
- [Endpoint en temps réel](https://learn.microsoft.com/fr-fr/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&source=recommendations&tabs=python)

### Dataset

TensorFlow met à disposition des datasets issues de directement de leur librarie. (tensorflow.keras.datasets)

Pour mettre tout ceci en pratique, on utilisera les données MNIST et on développera un modèle de Computer Vision avec TensorFlow afin de prédire le chiffre présent sur les images parmi 10 possibles : 0, 1, 2, 3, 4, 5, 6, 7, 8, 9.

# Mise en place de l'environnement

### Installer les librairies nécessaires pour intéragir avec Azure ML et autres


In [ ]:
!pip install -r requirements.txt

### Variables de connexion à Azure ML

In [ ]:
SUBSCRIPTION_ID = "_____" 
RESOURCE_GROUP = "_____"
WORKSPACE_NAME = "_____"

#### Connexion au workspace Azure ML pour utiliser le SDK Python Azure

Le SDK python permet d'utiliser l'API d'Azure avec du code plutôt qu'avec l'interface utilisateur.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(DefaultAzureCredential(), SUBSCRIPTION_ID, RESOURCE_GROUP, WORKSPACE_NAME)

### Import des librairies python

# PART 1: Entraînement d'un modèle

Nous allons maintenant créer un modèle de DL et nous l'entraînerons sur les données MNIST. Ce sont des images de digit en niveau de gris.

#### Script d'entrainement

Le script d'entraînement est dans src/tf_mnist.py

### Créer une ressource de calcul

Nous allons maintenant définir un job d'entraînement une ressource de calcul. Azure ML a besoin de cette dernière pour lancer des entraînements.

On provisionne ici un cluster de calcul basé sur Linux. Il y a de nombreuses séries de cluster, utilisant des CPUs, d'autres des GPUs, une certaine quantité de mémoire, de RAM...
Toutes ces caractéristiques ont aussi un rapport avec le prix horaire quand on loue le cluster. [Pricing cluster de calcul](https://azure.microsoft.com/fr-fr/pricing/details/machine-learning/)

Nous allons entraîner un modèle de Deep Learning, l'idéal est d'avoir du GPU, pour un entraînement efficace et rapide. Si, c'est possible (cela dépend de quota de coeurs GPU attribué à l'abonnement Azure), nous allons créer un cluster de taille NC8asT4v3. Si on a pas de coeur GPU, nous allons choisir le cluster suivant : STANDARD_F72s_v2

On utilise l'objet `AmlCompute` qui prend les paramètres suivant :
- name : Nom du cluster dans Azure ML (doit être unique dans le workspace).
- type : Type de ressource (toujours "amlcompute" pour ce type de cluster).
- size : Taille de la VM utilisé
- min_instances : Nombre minimal de nœuds actifs (0 signifie que le cluster peut s'arrêter complètement quand il est inactif pour économiser des coûts).
- max_instances : Nombre maximal de nœuds que le cluster peut scaler (ici, 4 VM maximum en parallèle)
- idle_time_before_scale_down : Délai en secondes avant de rendre le cluster inactif.
- tier : Priorité de la VM, 2 choix : "Dedicated" pour une VM dédiée (plus chères, garanties jusqu'à la fin du job) et "LowPriority" : Moins cher (~5x moins cher), mais risque d'interruption

In [ ]:
from azure.ai.ml.entities import AmlCompute

gpu_cluster = AmlCompute(
    name="_____",
    type="amlcompute",
    size="STANDARD_F72s_v2", 
    min_instances=0,
    max_instances=1,
    idle_time_before_scale_down=180,
    tier="LowPriority",
)

# Créer le cluster
gpu_cluster = ml_client.begin_create_or_update(gpu_cluster).result()

Veuillez vérifier que le cluster est bien crée en ouvrant le portail Azure ML sur votre navigateur. Il doit avoir 0 noeud de calcul.

## Configurer et soumettre l'entraînement du modèle

### Obtenir les données d'entraînement

In [ ]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
import pandas as pd

mnist_df = pd.DataFrame(X_train.reshape(X_train.shape[0], -1))
mnist_df['label'] = y_train

Représentation du dataframe :

In [ ]:
mnist_df

In [ ]:
mnist_df_test = pd.DataFrame(X_test.reshape(X_test.shape[0], -1))
mnist_df_test

In [ ]:
y_train

## Visualisation des données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 2. Sélectionner 10 images aléatoires
indices = np.random.choice(len(X_train), 10, replace=False)
sample_images = X_train[indices]
sample_labels = y_train[indices]

# 3. Visualisation des images (optionnel)
plt.figure(figsize=(10, 3))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(sample_images[i], cmap='gray')
    plt.title(f"Label: {sample_labels[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

## Processing des données pour l'entraînement

Ce script Python permet de convertir des images et leurs étiquettes dans le format IDX, un format binaire couramment utilisé pour les datasets d'entraînement comme MNIST

In [ ]:
import struct
import gzip
import os

# Fonction pour sauvegarder les images au format IDX
def save_images(images, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with gzip.open(filename, 'wb') as f:
        # Écrire l'en-tête
        magic_number = 2051  # pour les images
        num_images = len(images)
        num_rows = images.shape[1]
        num_cols = images.shape[2]
        
        f.write(struct.pack('>IIII', magic_number, num_images, num_rows, num_cols))
        
        # Écrire les données
        f.write(images.astype(np.uint8).tobytes())

# Fonction pour sauvegarder les étiquettes au format IDX
def save_labels(labels, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with gzip.open(filename, 'wb') as f:
        # Écrire l'en-tête
        magic_number = 2049  # pour les étiquettes
        num_items = len(labels)
        
        f.write(struct.pack('>II', magic_number, num_items))
        
        # Écrire les données
        f.write(labels.astype(np.uint8).tobytes())

## Sauvegarder les données d'entraînement et de test

Le nom des fichiers est nécessiare ensuite pour le script d'entraînement. Exécuter simplement la cellule.

In [ ]:
save_images(X_train, "./mnist_gzip/mnist/raw/train-images-idx3-ubyte.gz")
save_labels(y_train, "./mnist_gzip/mnist/raw/train-labels-idx1-ubyte.gz")
save_images(X_test, "./mnist_gzip/mnist/raw/t10k-images-idx3-ubyte.gz")
save_labels(y_test, "./mnist_gzip/mnist/raw/t10k-labels-idx1-ubyte.gz")

## Enregistrer le jeu de données Azure ML

Nous allons utiliser la section "Données" d'Azure Machine Learning pour stocker, gérer et versionner les données utilisés pour l'entraînement, la validation et l'inférence de modèles ML

On utilise l'objet `Data` qui prend les paramètres suivant :
- name : Nom que l'on donne à notre répertoire de données
- type : Type d'asset
- path : Chemin dans notre local ou se trouve nos données
- description : Phrase qui décrit nos données

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

mnist_data = Data(
    path="./mnist_gzip",
    type=AssetTypes.URI_FOLDER,
    name="mnist_gzip_dataset",
    description="MNIST en gzip"
)

ml_client.data.create_or_update(mnist_data)

## Créer un environnement pour le job

Un envrionnement est un conteneur qui encapsule toutes les dépendances (librairies Python, packages, variables d’environnement et configurations) nécessaires pour exécuter un script

Certains sont fournis directement par Azure Machine Learning, on va en utiliser un qui inclut tensorflow : "AzureML-tensorflow-2.12-cuda11@latest"

In [ ]:
curated_env_name = "AzureML-tensorflow-2.12-cuda11@latest"

## Entraînement

L'entraîenment se fait via un script (script d'entraînement) qui est ici : src/tf_mnist.py et a été réalisé au préalable.

La fonctionne `command` est utilisé ici, elle prend en argument :
- inputs : Dictionnaire qui définit toutes les entrées du job
  - data_folder : Chemin vers les données MNIST
  - batch_size : Taille des lots pour l'entraînement (on prendra 64)
  - first_layer_neurons : Nombre de neurones dans la première couche (on prendra 256)
  - second_layer_neurons : Nombre de neurones dans la seconde couche (on prendra 128)
  - learning_rate : Taux d'apprentissage du modèle (on prendra 0.01)
- compute : Nom du cluster GPU qui exécutera le job d'entraînement (créer précédemment)
- environment : Environnement d'exécution préconfiguré (curated_env_name)
- code : Chemin vers le répertoire contenant le code source ("./src/")
- command : Commande à exécuter avec les paramètres d'entrée
- experiment_name : Nom de l'expérience pour organiser les jobs dans Azure ML
- display_name : Nom du job affiché dans l'interface Azure ML

paramètre à renseigner ?

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input

job = command(
    inputs=dict(
        data_folder=Input(type="uri_folder", path=mnist_data.path),
        batch_size=__, # à remplir 
        first_layer_neurons=__, # à remplir 
        second_layer_neurons=__, # à remplir 
        learning_rate=__, # à remplir 
    ),
    compute=gpu_cluster.name,
    environment=curated_env_name,
    code="./src/",
    command="python tf_mnist.py --data-folder ${{inputs.data_folder}} --batch-size ${{inputs.batch_size}} --first-layer-neurons ${{inputs.first_layer_neurons}} --second-layer-neurons ${{inputs.second_layer_neurons}} --learning-rate ${{inputs.learning_rate}}",
    experiment_name="_____",
    display_name="_____",
)

## Lancer l'entraînement

In [ ]:
returned_job = ml_client.jobs.create_or_update(job)

Que se passe-t-il pendant l'exécution d'un entraînement sur Azure ?
Lors de son exécution, le travail passe par les étapes suivantes :

Préparation : Une image docker est créée en fonction de l'environnement défini. L'image est téléchargée dans le registre des conteneurs de le workspace.e

Mise à l'échelle : Le cluster "s'active" et devient disponible pour exécuter la tâche, il prend alors un noeud de calcul.

Exécution : Tous les scripts du dossier de scripts src sont téléchargés, les données sont copiées, et le script est exécuté. Les sorties de stdout et du dossier ./logs sont transmises à l'historique du travail et peuvent être utilisées pour surveiller le travail.

=> Temps d'entraînement : 9min (sans GPU)

## Tuning des hyperparamètres

Maintenant, voyons si on peut améliorer la précision de notre modèle. On peut régler et optimiser les hyperparamètres du modèle à l'aide du balayage d'Azure ML.

Pour ajuster les hyperparamètres du modèle, il faut définir l'espace de paramètres dans lequel on souhaite effectuer des recherches pendant l'entraînement. Pour ce faire, il faut remplacer certains des paramètres (batch_size, first_layer_neurons, second_layer_neurons et learning_rate) transmis à l'entraînement par des entrées spéciales du package azure.ml.sweep.

In [ ]:
from azure.ai.ml.sweep import Choice, LogUniform

job_for_sweep = job(
    batch_size=Choice(values=[32, 64, 128]), # Choix parmis 3 valeurs possibles
    first_layer_neurons=Choice(values=[16, 64, 128, 256, 512]), # Choix parmis 5 valeurs possibles
    second_layer_neurons=Choice(values=[16, 64, 256, 512]), # Choix parmis 4 valeurs possibles
    learning_rate=LogUniform(min_value=-6, max_value=-1), # Distribution uniforme logarithmique 10^-6 et 10^-1
)

On utilise la fonction `sweep` qui va prendre en argument:
- compute : Cluster utilisé pour l'entraînement
- sampling_algorithm :  Détermine la méthode d'échantillonnage des hyperparamètres
- primary_metric : Indique la métrique principale à optimiser pendant le balayage (ici on veut optimiser la métrique "validation_acc" qui est la précision sur l'ensemble de validation)
- goal : Spécifie si la primary_metric doit être maximisée ou minimisée
- max_total_trials :  Nombre maximum total d'expériences à exécuter
- max_concurrent_trials : Nombre maximum d'expériences pouvant s'exécuter en parallèle
- early_termination_policy : Configure la politique d'arrêt anticipé des essais non prometteurs (ici on utilisera BanditPolicy)

La politique Bandit utilise : 
- slack_factor : Seuil de tolérance par rapport au meilleur essai (on prendra 10% donc on arretera les essais si leurs performances sont inférieures de plus de 10% au meilleur essai)
- evaluation_interval : Fréquence d'évaluation de la politique d'arrêt anticipé (on prendra 2)

In [ ]:
from azure.ai.ml.sweep import BanditPolicy

sweep_job = job_for_sweep.sweep(
    compute=gpu_cluster.name,
    sampling_algorithm="random",
    primary_metric="validation_acc",
    goal="Maximize",
    max_total_trials=8,
    max_concurrent_trials=4,
    early_termination_policy=BanditPolicy(slack_factor=0.1, evaluation_interval=2),
)

## Lancement des entraînements

Temps d'exécution : ~ 20/25 min

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)

## Arrêter le cluster de calcul

Comme le cluster est facturé au temps d'utilisation, pensez à l'éteindre dès qu'il n'est plus nécessaire pour limiter les dépenses.

In [ ]:
ml_client.compute.begin_delete(gpu_cluster.name)

## Inscrire dans Azure le modèle entrainé

En allant sur le portail Azure Machine Learning, vous trouverez facilement, l'entraînement qui a produit le meilleur modèle. Il faut récupérer le nom de cet entraînement.

In [ ]:
# Nom du job ayant produit le meilleur modèle
run =  "_____"

On va utiliser l'objet `Model` pour enregistrer notre modèle, il comprend comme argument :
- path : Spécifie le chemin où le modèle entraîné est stocké dans Azure ML (ici il est dans une output d'entraînement)
- name : Nom unique pour le modèle
- description : Description du modèle
- type : Spécifie le type de modèle à enregistrer (ici c'est un "custom_model")

In [ ]:
from azure.ai.ml.entities import Model

# Enregistrement du modèle
model = Model(
    path=f"azureml://jobs/{run}/outputs/artifacts/paths/outputs/model/",
    name="_____", 
    description="_____",
    type="custom_model",
)

## Inscription du modèle dans Azure

In [ ]:
registered_model = ml_client.models.create_or_update(model=model)

# PART 2 : Déploiement en temps réel

## Inférence en temps réel

Nous allons maintenant utiliser le modèle pour faire de la prédiction en temps réel. Il faut déployer ce modèle sur un endpoint.

L'endpoint est un serveur qui tourne sur les ressources qu'on définit au préalable. Il gère les requêtes qu'il reçoit au format HTTP et distribue les calculs pour une réponse avec un minimum de latence. On expose donc un modèle via une API, permettant aux applications d'envoyer des données et de recevoir des prédictions.

Dans un premier temps, on va créer un endpoint de temps réel : `ManagedOnlineEndpoint`, qui comprend :
- name : Nom de l'endpoint
- description : Description de l'endpoint
- auth_mode : Mode d'authentification pour sécuriser l'accès à l'endpoint ("token" ou "key")

### Création d'un endpoint

In [ ]:
online_endpoint_name = "_____"

In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="_____",
    auth_mode="key",
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

On peut vérifier la bonne création de l'endpoint via code ou via le portail Azure ML.

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved')

## Déployer le modèle sur l'endpoint

Un déploiement sur un endpoint Azure ML est l'allocation d'une VM qui héberge votre modèle et le rend accessible via une API pour inférence.

Pour le déploiement, on a besoin d'un script de scoring. Il est ici : src/score.py
Le script de scoring comprend 2 fonctions : la fonction `init` qui lance le modèle, et la fonction `run` qui définit comment le modèle doit processer les données d'entrée quand on fait un appel à l'endpoint, et quelles doit être la sortie du modèle.

Pour le déploiement on utilise l'objet `ManagedOnlineDeployment`, qui prend en argument :
- name : Nom unique du déploiement dans l'endpoint
- endpoint_name : Nom de l'endpoint associé
- model : Nom du modèle enregistré à déployer
- code_configuration : Définit le code source et le script de scoring pour l'inférence
- environment : Environnement d'exécution contenant les dépendances nécessaires (on utilisera la mêm que l'entraînement)
- instance_type : Taille de la VM utilisée pour héberger le modèle
- instance_count : Nombre d'instances pour le scaling horizontal (on en utilisera qu'une)

Temps de déploiement : ~5 min

In [ ]:
model = registered_model

from azure.ai.ml.entities import CodeConfiguration

# create an online deployment.
deployment = ManagedOnlineDeployment(
    name="_____",
    endpoint_name=online_endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(code="./src", scoring_script="score.py"),
    environment=curated_env_name,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

deployment = ml_client.begin_create_or_update(deployment).result()

Après avoir déployé un modèle sur un endpoint Azure ML, le trafic est initialement à 0% par défaut. Cela signifie qu'aucune requête ne sera routée vers le nouveau déploiement. On peut manuellement mettre le traffic de l'endpoint à 100% sur le portail Azure ou via du code :

In [ ]:
# Créer une règle de trafic pour le nouveau déploiement
traffic_rules = {"_____": 100}

# Mettre à jour l'endpoint avec les nouvelles règles de trafic
endpoint.traffic = traffic_rules
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Q1 : Quel est le coût de la ressource "Standard_DS3_v2" à l'heure ?

## Test du modèle

Maintenant que le modèle est entraîné et déployer sur un endpoint on veut vérifier son bon fonctionnement.

Dans le repertoire request, il y a un fichier au format json (sample-request) qui associe la clé "data" à une liste comprenant une liste de 784 valeurs représentant une image MNIST en niveau de gris.

On va envoyer ce json à l'endpoint et on devrait recevoir le résultat.

Pour vérifier la bonne réponse du modèle, observons d'abord à quelle valeur est associé l'image.

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

# 1. Charger le fichier JSON
with open('request\sample-request.json') as f:
    data = json.load(f)

# 2. Extraire les tableaux de pixels
images_data = data['data']  # Liste de tableaux de 784 valeurs

# 3. Convertir en format image (28x28)
images = [np.array(img).reshape(28, 28) for img in images_data]

# 4. Affichage avec matplotlib
plt.figure(figsize=(15, 5))
for i, (img, img_data) in enumerate(zip(images, images_data)):
    plt.subplot(1, len(images), i+1)
    plt.imshow(img, cmap='gray')
    plt.axis('off')

plt.tight_layout()
plt.show()

## Lancement test

In [ ]:
result = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./request/sample-request.json",
    deployment_name="_____",
)

In [ ]:
print(result)

# Inférence par lot

Nous allons maintenant utiliser le modèle pour faire de la prédiction par lot (batch).

Le déploiement batch traite des volumes de données en asynchrone (idéal pour les gros fichiers), tandis que le déploiement temps réel répond instantanément aux requêtes individuelles (pour les applications interactives)

Le fonctionnement du déploiement est différent pour l'inférence par lot :
- l'endpoint doit être un `BatchEndpoint`
- il faut lancer un cluster de calcul via `AmlCompute` à utiliser
- il faut créer un environnement personnalisé avec l'objet `Environment`

On va alors créer un objet `BatchEndpoint`, qui prend en argument :
- name : Nom de l'endpoint
- description : Description de l'endpoint

In [ ]:
batch_endpoint_name = "_____"

In [ ]:
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment

# Créer un endpoint batch
batch_endpoint = BatchEndpoint(
    name=batch_endpoint_name,
    description="Processing en lot des images MNIST"
)
ml_client.begin_create_or_update(batch_endpoint).result()

Création d'un cluster de calcul

In [ ]:
from azure.ai.ml.entities import AmlCompute

cpu_compute = AmlCompute(
    name="_____",
    size="STANDARD_DS3_V2",
    min_instances=0,
    max_instances=2
)
ml_client.compute.begin_create_or_update(cpu_compute)

Créer l'environnement

In [ ]:
from azure.ai.ml.entities import Environment

# Créer un environnement à partir de l'image Docker
env = Environment(
    name="_____",
    image="mcr.microsoft.com/azureml/curated/tensorflow-2.16-cuda12:11",
    description="Environnement TensorFlow pour le traitement batch"
)
    
# Créer ou mettre à jour l'environnement dans Azure ML
env = ml_client.environments.create_or_update(env)

## Déploiement

On va utiliser l'objet `BatchDeployment` qui prend les mêmes arguments que `ManagedOnlineDeployment`. Cependant, on utilise un script de scoring spécial pour l'inférence par lot.


In [ ]:
batch_deployment = BatchDeployment(
    name="_____",
    endpoint_name=batch_endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(code="./src", scoring_script="batch_score.py"),
    environment=env,
    compute=cpu_compute.name,  # Cluster dédié
    instance_count=2
)
ml_client.begin_create_or_update(batch_deployment).result()

La différence entre de l'inférence par lot et par batch est qu'on doit ajouter les données à prédire dans Azure ML avec l'objet `Data`. Les données sont les 5 json du folder batch_request.

Ils contiennent tous une clé "data" associé à une liste de 30 listes comprenant 784 valeurs, représentant des images MNIST en niveau de gris.
Il y 30 images associé à "data", soit un total de 150 images à analyser par le modèle.

In [ ]:
input_data_asset = Data(
    path="./batch_request/",
    type=AssetTypes.URI_FOLDER,
    name="_____",
    description="Données en lot d'images MNIST",
)
ml_client.data.create_or_update(input_data_asset)

Après avoir créer ses données on les utilise comme un `Input`

In [ ]:
data_asset = ml_client.data.get(name=input_data_asset.name, label="latest")
input_data_batch = Input(path=data_asset.id)

## Lancement du test par lot

Ce test va lancer un job dans Azure qui va classifier toutes nos images. L'éxécution prend en moyenne 12 minutes.

In [ ]:
# Créer un job batch
result_batch = ml_client.batch_endpoints.invoke(
    endpoint_name=batch_endpoint_name,
    deployment_name=batch_deployment.name,
    inputs={"data":input_data_batch}
)

In [ ]:
ml_client.jobs.get(result_batch.name)

### Récupération des résultats dans un fichiers predictions.csv en les téléchargeant depuis Azure

In [ ]:
ml_client.jobs.download(name=result_batch.name, download_path=".", output_name="score")

## Evaluer les résulats

Dans le repertoire batch_request_result, il y a 5 json qui correspondent à la valeur réelle des images MNIST passées en prédiction.

Chaque json associe la clé "labels" à une liste comprenant 30 valeurs.

Les scripts suivant vont évaluer nos résultats !

In [ ]:
import json
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def load_json_file(file_path):
    """Load labels from a JSON file"""
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data["labels"]

def load_predictions(file_path):
    """Load predictions from CSV file"""
    predictions = []
    with open(file_path, 'r') as f:
        for line in f:
            list_line=[int(x) for x in line if x.isdigit()]
            predictions.extend(list_line)
        print(len(predictions))
    return predictions

def evaluate(y_true, y_pred):
    """Evaluate predictions against true labels"""
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    
    # Print classification report
    print("Classification Report:")
    print(classification_report(y_true, y_pred, digits=4))
    
    # Print confusion matrix
    print("Confusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

In [ ]:
true_labels_paths = ["batch_request_result\mnist_labels_batch_1.json", "batch_request_result\mnist_labels_batch_2.json", "batch_request_result\mnist_labels_batch_3.json",
"batch_request_result\mnist_labels_batch_4.json", "batch_request_result\mnist_labels_batch_5.json"]
predictions_path = "predictions.csv"

# Load true labels
true_labels = []
for path in true_labels_paths:
    true_labels.extend(load_json_file(path))

# Load predictions
predictions = load_predictions(predictions_path)

# Evaluate
print(f"Evaluating {len(true_labels)} samples...")
evaluate(true_labels, predictions)

### Arrêt du cluster et de l'endpoint par batch

In [ ]:
ml_client.compute.begin_delete(cpu_compute.name)

In [ ]:
ml_client.online_endpoints.begin_delete(
    name=batch_endpoint_name
)

# Partie 4 : Montée en charge de l'endpoint 

Dans cette partie nous allons tester la capacité de notre endpoint a pouvoir monter en charge. Dans un cas d'usage où nous constuisons une application ayant comme vocation a recevoir beacuoup de connexion, 
nous devons nous assurer que les modèles ont cette capacité a traiter ce traffic. 

Ainsi, nous allons utiliser la librairie Locust, elle permet de réaliser des test de charge sur des API. 

- Ouvrez le script locust.py vérifiez bien que la fonction load_data() fonctionne dans votre cas. Ensuite ajouter les variables nécessaires d'Azure;


- Lancer l'application locust avec la commande $locust -f locustfile.py (!! librairie nécessaire locust, PyYAML, flask !!) 

- Renseigner le nombre d'utilisateurs, leurs intervalle d'apparition ainsi que l'URL à surchargé (url de scrore à renseigner)

Réalisez plusieurs test de votre endpoint, vous pouvez cahgner la charge en augmentant le nombre d'utilisateurs faisant des requètes

Q2 : A partir de combien d'utilisateurs la réponse médiane met plus de 3 secondes à répondre ? Et à partir de combien d'utilisateurs, on atteint 5 % d'erreur ? 

Changeons la ressource sur laquelle le modèle est déployé, il faut aller dans Azure Machine Learning, passer le traffic du déploiement à 0, et enfin supprimer le déploiement. Passons à Standard_DS5_v2, vous pouvez le redéployer à la main directement su rle portail Azure, en choisissant le bon script de scoring, ou en remontant à la cellule du déploiement en temps réel, et en changeant le compute par "Standard_DS5_v2".

Q3 : Quelles sont les différences entre les deux VM ?

Q4 : Avec cette nouvelle VM : A partir de combien d'utilisateurs la réponse médiane met plus de 3 secondes à répondre ? Et à partir de combien d'utilisateurs, on atteint 5 % d'erreur ? 

Q5 : Comment pourrait-on améliorer les résultats de nos tests de charge ? Si vous aviez accès à a n'importe quelle VM, laquelle utiliseriez vous pour deployer votre modèle ? (Le coût est à prendre en compte)

# Nettoyage du projet

### Arrêt de l'endpoint en temps réel

In [ ]:
ml_client.online_endpoints.begin_delete(
    name=online_endpoint_name,
)